## Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import csv
import datetime as dt

### Bank and Prime Interest Rates

In [2]:
data_link = r"D:\UofT\Data Analytics Boot Camp\Final Project\GitFolder\Data-Analytics-Boot-Camp---Final-Project\Datasets\Financial market statistics\10100145.csv"
prime_interest_rate_df = pd.read_csv(data_link)
prime_interest_rate_df.head()

,REF_DATE,GEO,DGUID,Rates,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1957-01-02,Canada,2016A000011124,Bank rate,Percent,239,units,0,v80691310,1.2,NaN,..,NaN,NaN,2
1,1957-01-02,Canada,2016A000011124,Chartered bank administered interest rates - P...,Percent,239,units,0,v80691311,1.4,NaN,..,NaN,NaN,2
2,1957-01-02,Canada,2016A000011124,Chartered bank - consumer loan rate,Percent,239,units,0,v80691312,1.5,NaN,..,NaN,NaN,2
3,1957-01-02,Canada,2016A000011124,"Forward premium or discount (-), United States...",Percent,239,units,0,v80691313,1.6,NaN,..,NaN,t,2
4,1957-01-02,Canada,2016A000011124,"Forward premium or discount (-), United States...",Percent,239,units,0,v80691314,1.7,NaN,..,NaN,t,2


In [3]:
bank_rate_df = prime_interest_rate_df[prime_interest_rate_df["Rates"] == "Bank rate"]
bank_rate_df = bank_rate_df[["REF_DATE", "VALUE"]]
bank_rate_df = bank_rate_df.dropna()
bank_rate_df["REF_DATE"] = pd.to_datetime(bank_rate_df["REF_DATE"], infer_datetime_format=True)
bank_rate_df.columns = ["Date", "Bank_Rate"]

In [4]:
bank_rate_df_by_month = bank_rate_df.groupby(pd.Grouper(key="Date", freq="M")).mean()
bank_rate_df_by_month.index = bank_rate_df_by_month.index.strftime('%Y-%m')
bank_rate_df_by_month.head()

,Bank_Rate
Date,
1980-01,14.000
1980-02,14.000
1980-03,14.245
1980-04,16.046
1980-05,13.685


In [5]:
prime_rate_df = prime_interest_rate_df[prime_interest_rate_df["Rates"] == "Chartered bank administered interest rates - Prime rate"]
prime_rate_df = prime_rate_df[["REF_DATE", "VALUE"]]
prime_rate_df = prime_rate_df.dropna()
prime_rate_df["REF_DATE"] = pd.to_datetime(prime_rate_df["REF_DATE"], infer_datetime_format=True)
prime_rate_df.columns = ["Date", "Prime_Rate"]

In [6]:
prime_rate_df_by_month = prime_rate_df.groupby(pd.Grouper(key="Date", freq="M")).mean()
prime_rate_df_by_month.index = prime_rate_df_by_month.index.strftime('%Y-%m')
prime_rate_df_by_month.head()

,Prime_Rate
Date,
1975-01,10.7
1975-02,9.5
1975-03,9.0
1975-04,9.0
1975-05,9.0


### Inflation Rate

In [7]:
def season_to_month(x):
    x = x.split("Q", 2)
    year = x[0]
    quater = x[1]
    if str(quater) == "1":
        date = year + "-01"
        return date
    elif str(quater) == "2":
        date = year + "-04"
        return date  
    elif str(quater) == "3":
        date = year + "-07"
        return date      
    elif str(quater) == "4":
        date = year + "-10"
        return date     

In [8]:
data_link = r"D:\UofT\Data Analytics Boot Camp\Final Project\GitFolder\Data-Analytics-Boot-Camp---Final-Project\Datasets\Inflation.csv"
inflation_rate_df = pd.read_csv(data_link, skiprows=20,infer_datetime_format=True)
inflation_rate_df.head()

,date,INDINF_CPI_Q,INDINF_CPI_TRIM_Q,INDINF_CPI_MEDIAN_Q,INDINF_CPI_COMMON_Q
0,1993Q1,2.2,2.0,2.0,2.0
1,1993Q2,1.8,1.9,1.9,1.8
2,1993Q3,1.8,1.8,1.7,1.6
3,1993Q4,1.8,1.8,1.7,1.5
4,1994Q1,0.5,1.5,1.6,1.3


In [9]:
inflation_rate_df = inflation_rate_df[["date", "INDINF_CPI_Q"]]
inflation_rate_df["Date"] = inflation_rate_df["date"].apply(lambda x: season_to_month(x))
inflation_rate_df["Date"] = pd.to_datetime(inflation_rate_df["Date"], infer_datetime_format=True)
inflation_rate_df.index = inflation_rate_df["Date"]
inflation_rate_df.drop(columns = ["date","Date"], inplace=True)
inflation_rate_df.index = inflation_rate_df.index.strftime('%Y-%m')
inflation_rate_df.head()

,INDINF_CPI_Q
Date,
1993-01,2.2
1993-04,1.8
1993-07,1.8
1993-10,1.8
1994-01,0.5


### GDP

In [10]:
def year_to_month(x):
    return x + "-01"

In [11]:
data_link = r"D:\UofT\Data Analytics Boot Camp\Final Project\GitFolder\Data-Analytics-Boot-Camp---Final-Project\Datasets\gdp_population\API_CAN_DS2_en_csv_v2_2255524.csv"
GDP_df = pd.read_csv(data_link, skiprows=4)
GDP_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Canada,CAN,Battle-related deaths (number of people),VC.BTL.DETH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Canada,CAN,Merchandise exports to low- and middle-income ...,TX.VAL.MRCH.R1.ZS,5.544684e-01,2.392815e+00,2.618122e+00,1.813149e+00,1.979730e+00,1.552714e+00,...,5.186957e+00,5.306948e+00,4.576023e+00,4.835928e+00,4.899990e+00,5.252423e+00,5.763911e+00,4.849690e+00,NaN,NaN
2,Canada,CAN,Travel services (% of commercial service imports),TM.VAL.TRVL.ZS.WT,4.681692e+01,3.930260e+01,3.687604e+01,3.504129e+01,3.797121e+01,3.909896e+01,...,3.296319e+01,3.308219e+01,3.326790e+01,3.265973e+01,3.158360e+01,3.123187e+01,2.890012e+01,2.980368e+01,NaN,NaN
3,Canada,CAN,Merchandise imports (current US$),TM.VAL.MRCH.CD.WT,6.073000e+09,6.062000e+09,6.220000e+09,6.445000e+09,7.402000e+09,8.622000e+09,...,4.762960e+11,4.757770e+11,4.753190e+11,4.301240e+11,4.129400e+11,4.436510e+11,4.705220e+11,4.636630e+11,NaN,NaN
4,Canada,CAN,"Bound rate, simple mean, primary products (%)",TM.TAX.TCOM.BR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.740000e+00,6.670000e+00,6.650000e+00,2.720000e+00,2.720000e+00,2.730000e+00,2.730000e+00,2.730000e+00,NaN,NaN


In [12]:
GDP_df = GDP_df[GDP_df["Indicator Name"] == "GDP growth (annual %)"]
GDP_df = GDP_df.drop(columns=["Country Name", "Country Code", "Indicator Name", "Indicator Code"])
GDP_df = GDP_df.transpose()
GDP_df["Date"] = GDP_df.index
GDP_df["Date"] = GDP_df["Date"].apply(lambda x: year_to_month(x))
GDP_df.index = GDP_df["Date"]
GDP_df.drop(columns=["Date"], inplace=True)
GDP_df.columns=["GDP_Growth"]
GDP_df.drop("Unnamed: 65-01", axis=0, inplace=True)
GDP_df = GDP_df.dropna()
GDP_df.head()

,GDP_Growth
Date,
1961-01,3.163292
1962-01,7.425385
1963-01,5.349546
1964-01,6.642894
1965-01,6.316714


### Weekly BCPI Metals and Minerals

In [13]:
data_link = r"D:\UofT\Data Analytics Boot Camp\Final Project\GitFolder\Data-Analytics-Boot-Camp---Final-Project\Datasets\Commodity Price Index.csv"
metal_mineral_price_df = pd.read_csv(data_link, skiprows=20,infer_datetime_format=True)
metal_mineral_price_df.head()

,date,W.BCPI,W.BCNE,W.ENER,W.MTLS,W.FOPR,W.AGRI,W.FISH
0,1972-01-12,100.00,100.00,100.00,100.00,100.00,100.00,100.00
1,1972-01-19,100.00,100.00,100.00,100.00,100.00,100.00,100.00
2,1972-01-26,100.00,100.00,100.00,100.00,100.00,100.00,100.00
3,1972-02-02,100.16,100.21,99.91,100.29,100.05,100.48,95.56
4,1972-02-09,100.40,100.53,99.78,100.74,100.12,101.20,88.89


In [14]:
metal_mineral_price_df = metal_mineral_price_df[["date", "W.MTLS"]]
metal_mineral_price_df["Date"] = metal_mineral_price_df["date"]
metal_mineral_price_df["Date"] = pd.to_datetime(metal_mineral_price_df["Date"], infer_datetime_format=True)
metal_mineral_price_df.drop(columns = ["date"], inplace=True)

In [15]:
metal_mineral_price_df_by_month = metal_mineral_price_df.groupby(pd.Grouper(key="Date", freq="M")).mean()
metal_mineral_price_df_by_month.index = metal_mineral_price_df_by_month.index.strftime('%Y-%m')
metal_mineral_price_df_by_month.dropna(inplace=True)
metal_mineral_price_df_by_month.head()

,W.MTLS
Date,
1972-01,100.0000
1972-02,100.6275
1972-03,101.2780
1972-04,101.2275
1972-05,101.2480


### NASDAQ

In [16]:
data_link = r"D:\UofT\Data Analytics Boot Camp\Final Project\GitFolder\Data-Analytics-Boot-Camp---Final-Project\Datasets\NASDAQ.csv"
NASDAQ_df = pd.read_csv(data_link, infer_datetime_format=True)
NASDAQ_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1971-02-05,100.000000,100.000000,100.000000,100.000000,100.000000,0
1,1971-02-08,100.839996,100.839996,100.839996,100.839996,100.839996,0
2,1971-02-09,100.760002,100.760002,100.760002,100.760002,100.760002,0
3,1971-02-10,100.690002,100.690002,100.690002,100.690002,100.690002,0
4,1971-02-11,101.449997,101.449997,101.449997,101.449997,101.449997,0


In [17]:
NASDAQ_df = NASDAQ_df[["Date", "Open"]]
NASDAQ_df["Date"] = pd.to_datetime(NASDAQ_df["Date"], infer_datetime_format=True)
NASDAQ_df_by_month = NASDAQ_df.groupby(pd.Grouper(key="Date", freq="M")).mean()
NASDAQ_df_by_month.index = NASDAQ_df_by_month.index.strftime('%Y-%m')
NASDAQ_df_by_month.dropna(inplace=True)
NASDAQ_df_by_month.head()

,Open
Date,
1971-02,100.963333
1971-03,104.448260
1971-04,109.555238
1971-05,109.570500
1971-06,108.567728


### USD to CAD

In [18]:
data_link = r"D:\UofT\Data Analytics Boot Camp\Final Project\GitFolder\Data-Analytics-Boot-Camp---Final-Project\Datasets\us_dollar\WEBSTATS_XRU_CURRENT_D_DATAFLOW_csv_row.csv"
USD_df = pd.read_csv(data_link, infer_datetime_format=True)
USD_df.head()

C:\Users\Jesse_Desktop\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Frequency,D:Daily,D:Daily.1,D:Daily.2,D:Daily.3,D:Daily.4,D:Daily.5,D:Daily.6,D:Daily.7,D:Daily.8,...,D:Daily.71,D:Daily.72,D:Daily.73,D:Daily.74,D:Daily.75,D:Daily.76,D:Daily.77,D:Daily.78,D:Daily.79,D:Daily.80
0,Reference area,AE:United Arab Emirates,AL:Albania,AR:Argentina,AT:Austria,AU:Australia,BA:Bosnia and Herzegovina,BE:Belgium,BG:Bulgaria,BH:Bahrain,...,TR:Turkey,TT:Trinidad and Tobago,TW:Chinese Taipei,UA:Ukraine,US:United States,UY:Uruguay,VE:Venezuela,XM:Euro area,XW:World,ZA:South Africa
1,Currency,AED:UAE Dirham,ALL:Albanian Lek,ARS:Argentine Peso,EUR:Euro,AUD:Australian Dollar,BAM:Bosnian Convertible Mark,EUR:Euro,BGN:Bulgarian Lev,BHD:Bahraini Dinar,...,TRY:Turkish Lira,TTD:Trinidad and Tobago Dollar,TWD:New Taiwan Dollar,UAH:Ukrainian Hryvnia,USD:US Dollar,UYU:Uruguayan Peso,VEF:Bolivar Fuerte,EUR:Euro,XDR:SDR,ZAR:South African Rand
2,Collection,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,...,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period,A:Average of observations through period
3,Time Period,D:AE:AED:A,D:AL:ALL:A,D:AR:ARS:A,D:AT:EUR:A,D:AU:AUD:A,D:BA:BAM:A,D:BE:EUR:A,D:BG:BGN:A,D:BH:BHD:A,...,D:TR:TRY:A,D:TT:TTD:A,D:TW:TWD:A,D:UA:UAH:A,D:US:USD:A,D:UY:UYU:A,D:VE:VEF:A,D:XM:EUR:A,D:XW:XDR:A,D:ZA:ZAR:A
4,1949-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN


In [19]:
USD_df = USD_df[["Frequency", "D:Daily.11"]]
USD_df = USD_df.drop(USD_df.index[0:4], axis=0)
USD_df.columns = ["Date", "Exchange Rate"]
USD_df.dropna(inplace=True)
USD_df["Date"] = pd.to_datetime(USD_df["Date"], infer_datetime_format=True)
USD_df["Exchange Rate"] = USD_df["Exchange Rate"].astype("float64")
USD_df["Date"] = pd.to_datetime(USD_df["Date"], infer_datetime_format=True)
USD_df_by_month = USD_df.groupby(pd.Grouper(key="Date", freq="M")).mean()
USD_df_by_month.index = USD_df_by_month.index.strftime('%Y-%m')
USD_df_by_month.head()

,Exchange Rate
Date,
1950-10,1.053333
1950-11,1.040318
1950-12,1.053075
1951-01,1.051873
1951-02,1.049130


### Gold Price

In [20]:
data_link = r"D:\UofT\Data Analytics Boot Camp\Final Project\GitFolder\Data-Analytics-Boot-Camp---Final-Project\Datasets\gold.csv"
gold_df = pd.read_csv(data_link, infer_datetime_format=False)
gold_df.head()

C:\Users\Jesse_Desktop\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (19,21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Market,Perth Mint Spot,Perth Mint Spot.1,Perth Mint Spot.2,Perth Mint Spot.3,Perth Mint Spot.4,Perth Mint Spot.5,Perth Mint Spot.6,Perth Mint Spot.7,Perth Mint Spot.8,...,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98
0,Metal/Currency,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Priced In,USD,USD,AUD,AUD,JPY,JPY,USD,USD,AUD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Price Type,Bid,Ask,Bid,Ask,Bid,Ask,Bid,Ask,Bid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Summary,High,High,High,High,High,High,Low,Low,Low,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/01/68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
gold_df = gold_df[["Market", "Perth Mint Spot.12"]]
gold_df.columns = ["Date", "Gold_Price_in_USD"]
gold_df = gold_df.drop(gold_df.index[0:4], axis=0)
gold_df.dropna(inplace=True)
gold_df["Date"] = pd.to_datetime(gold_df["Date"], format="%d/%m/%y")
gold_df["Gold_Price_in_USD"] = gold_df.iloc[:,1].str.replace(',', '').astype(float)
gold_df["Gold_Price_in_USD"] = gold_df["Gold_Price_in_USD"].astype("float64")
gold_df_by_month = gold_df.groupby(pd.Grouper(key="Date", freq="M")).mean()
gold_df_by_month.index = gold_df_by_month.index.strftime('%Y-%m')
gold_df_by_month.head()

,Gold_Price_in_USD
Date,
1991-06,365.887500
1991-07,368.015185
1991-08,356.464074
1991-09,348.186800
1991-10,358.873704


### Gold Price after 2016

In [22]:
data_link = r"D:\UofT\Data Analytics Boot Camp\Final Project\GitFolder\Data-Analytics-Boot-Camp---Final-Project\Datasets\gold-Current.csv"
gold_df_current = pd.read_csv(data_link, infer_datetime_format=False)
gold_df_current.head()

,Market,Perth Mint Spot,Perth Mint Spot.1,Perth Mint Spot.2,Perth Mint Spot.3,Perth Mint Spot.4,Perth Mint Spot.5,Perth Mint Spot.6,Perth Mint Spot.7,Perth Mint Spot.8,Perth Mint Spot.9,Perth Mint Spot.10,Perth Mint Spot.11,Perth Mint Spot.12,Perth Mint Spot.13,Perth Mint Spot.14,Perth Mint Spot.15,Perth Mint Spot.16,Perth Mint Spot.17,Lease Rates
0,Metal/Currency,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,Gold,6 Month
1,Priced In,USD,USD,AUD,AUD,JPY,JPY,USD,USD,AUD,AUD,JPY,JPY,USD,USD,AUD,AUD,JPY,JPY,Gold
2,Price Type,Bid,Ask,Bid,Ask,Bid,Ask,Bid,Ask,Bid,Ask,Bid,Ask,Bid,Ask,Bid,Ask,Bid,Ask,% pa
3,Summary,High,High,High,High,High,High,Low,Low,Low,Low,Low,Low,Average,Average,Average,Average,Average,Average,NaN
4,01/01/16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
gold_df_current = gold_df_current[["Market", "Perth Mint Spot.12"]]
gold_df_current.columns = ["Date", "Gold_Price_in_USD_after_2016"]
gold_df_current = gold_df_current.drop(gold_df_current.index[0:4], axis=0)
gold_df_current.dropna(inplace=True)
gold_df_current["Date"] = pd.to_datetime(gold_df_current["Date"], format="%d/%m/%y")
gold_df_current["Gold_Price_in_USD_after_2016"] = gold_df_current.iloc[:,1].str.replace(',', '').astype(float)
gold_df_current["Gold_Price_in_USD_after_2016"] = gold_df_current["Gold_Price_in_USD_after_2016"].astype("float64")
gold_df_current_by_month = gold_df_current.groupby(pd.Grouper(key="Date", freq="M")).mean()
gold_df_current_by_month.index = gold_df_current_by_month.index.strftime('%Y-%m')
gold_df_current_by_month.head()

,Gold_Price_in_USD_after_2016
Date,
2016-01,1090.861579
2016-02,1190.085714
2016-03,1240.928500
2016-04,1236.149000
2016-05,1255.700909


In [24]:
gold_df_current_by_month.index.value_counts()

2021-03    1
2018-05    1
2017-09    1
2017-01    1
2019-01    1
          ..
2018-01    1
2019-03    1
2021-02    1
2017-04    1
2017-11    1
Name: Date, Length: 64, dtype: int64

### Merge

In [25]:
Merged = USD_df_by_month.join([prime_rate_df_by_month, inflation_rate_df, GDP_df, metal_mineral_price_df_by_month, NASDAQ_df_by_month, gold_df_by_month])

In [26]:
Merged.to_csv(r"D:\UofT\Data Analytics Boot Camp\Final Project\GitFolder\Data-Analytics-Boot-Camp---Final-Project\Datasets\Jesse_Cleaned_Data.csv")